In [ ]:
from google.colab import drive
drive.mount("/content/mydrive/")
%cd /content/mydrive/MyDrive/LTSF_Linear

import os
import json
from glob import glob

import numpy as np
import pandas as pd

from scipy.stats import levene, bartlett
from scipy import stats

root_path = "dataset/Water_Week"
dir_list = sorted(os.listdir(root_path))
dir_list

Drive already mounted at /content/mydrive/; to attempt to forcibly remount, call drive.mount("/content/mydrive/", force_remount=True).
/content/mydrive/MyDrive/LTSF_Linear


['C001팔당2취',
 'D001청주정',
 'D002구미',
 'D003고령',
 'D004주암',
 'E003본포',
 'E004남강']

In [ ]:
def get_ydata(root_path: str, data_path: str, ):
    """_summary_
    Args:
        root_path (str): _description_
        data_path (str): _description_
    Returns:
        list, list, pd.DataFrame: _description_
    """
    label_list = sorted(glob(os.path.join(root_path, data_path, "hw3_json/*.json")))
    target_cols = ['turbidity', 'EC', 'pH', 'water_temp',
        'DO', 'TOC', 'algae', 'alkalinity', 'blue_algae', 'residual_Cl',
        'blue-green_algae', 'diatomeae', 'cryptophyceae', '2-MIB', 'Geosmin',
        'synedra', 'T-N', 'T-P', 'Mn',]
    target_vals = []
    for js in label_list:
        with open(js, 'r') as f:
            label = json.load(f)
        input_cols = label['h']['input_serial'].split(',')   
        target_cols = [k for k,v in label['w'].items() if (v is not None) & (k in target_cols)]
        vals = [label['w'][k] for k in target_cols]
        vals = [v.replace(',','') if str(v).__contains__(',') else v for v in vals ]
        target_vals.append(vals)
    target_vals = np.array(target_vals)    
    return input_cols, target_cols, pd.DataFrame(target_vals, columns = target_cols, dtype=float)

### 2-MIB 기준 turbidity A

In [ ]:
print(dir_list[1])
input_cols, target_cols, df_target = get_ydata(root_path, dir_list[1])
mean_target = df_target["2-MIB"].mean()
index_list = list(df_target[df_target["2-MIB"] > mean_target].index)

data_list = sorted(glob(os.path.join(root_path, dir_list[1], "csv/*.csv")))
cnt = 0
path_list = []
for file in data_list:
    if cnt in index_list:
        path_list.append(file)
    cnt += 1
#path_list

df_raw = pd.DataFrame()
for file in path_list:
    df_ = pd.read_csv(file)
    df_raw = pd.concat([df_raw, df_], axis = 0, ignore_index= True)
#df_raw


x_data = df_raw[df_raw.columns.intersection(input_cols)]
turbidityA = x_data["turbidity"].values # gropA
#x_data.describe().loc["mean", :] # gropA의 컬럼별 평균
turbidityA


D001청주정


array([1.4734, 1.4464, 1.3995, ..., 3.7339, 3.7828, 3.9   ])

### 2-MIB 기준 turbidity B

In [ ]:
print(dir_list[1])
input_cols, target_cols, df_target = get_ydata(root_path, dir_list[1])
mean_target = df_target["2-MIB"].mean()
index_list = list(df_target[df_target["2-MIB"] <= mean_target].index)

data_list = sorted(glob(os.path.join(root_path, dir_list[1], "csv/*.csv")))
cnt = 0
path_list = []
for file in data_list:
    if cnt in index_list:
        path_list.append(file)
    cnt += 1
#path_list

df_raw = pd.DataFrame()
for file in path_list:
    df_ = pd.read_csv(file)
    df_raw = pd.concat([df_raw, df_], axis = 0, ignore_index= True)
#df_raw


x_data = df_raw[df_raw.columns.intersection(input_cols)]
turbidityB = x_data["turbidity"].values # gropA
#x_data.describe().loc["mean", :] # gropA의 컬럼별 평균
turbidityB


D001청주정


array([1.6656, 1.637 , 1.6422, ..., 3.0677, 3.0245, 2.6308])

In [ ]:
# 귀무가설: turbidityA와 turbidityB 두 집단에 따른 turbidity는 차이가 없다.
# 대립가설: turbidityA와 turbidityB 두 집단에 따른 turbidity는 차이가 있다.

In [ ]:
# 두 집단의 평균 차이
np.mean(turbidityA), np.mean(turbidityB)

(6.486008177083333, 3.8944526746532895)

In [ ]:
# 등분산성 검정: 방법1
levene(turbidityA, turbidityB)

LeveneResult(statistic=762.9094220175293, pvalue=7.305935552886479e-163)

In [ ]:
# 등분산성 검정: 방법2
bartlett(turbidityA, turbidityB)

BartlettResult(statistic=2059.7461032840424, pvalue=0.0)

In [ ]:
# 등분산성 검정: 두 방법의 p-value가 유의수준 0.05 보다 더 작다. 
#                따라서 두 집단의 데이터는 등분산성을 만족한다고 볼 수 없다.

In [ ]:
# 독립표본 t-검정 시행: 등분산성을 만족하지 않으므로, equal_var=False로 설정.
stats.ttest_ind(turbidityA, turbidityB, equal_var=False)

Ttest_indResult(statistic=24.380778344213198, pvalue=3.7008652086106905e-121)

- **잠정적 결론1** : p<0.05 이므로 두 집단의 평균 차이는 통계적으로 유의미하다.
- **잠정적 결론2** : p<0.05 이므로 귀무가설을 기각한다.따라서 turbidityA와 turbidityB 두 집단에 따른 turbidity는 통계적으로 유의한 차이가 존재한다.


In [ ]:
"""
for col in x_data.columns:
    print(f"{col} : {x_data.loc[:, col].values}")
"""

'\nfor col in x_data.columns:\n    print(f"{col} : {x_data.loc[:, col].values}")\n'